In [ ]:
### Importing Libraries ###

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"


In [ ]:
### Google Drive ###

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/RFM Analysis (1)/rfm_data.csv")
data=pd.DataFrame(df)

In [ ]:
data

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


**Calculating RFM Scores/Values**

In [ ]:
from datetime import datetime

In [ ]:
## Convert 'PurchaseDate' to datetime ##

data['PurchaseDate']=pd.to_datetime(data['PurchaseDate'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 47.0+ KB


In [ ]:
# # Format the PurchaseDate to DD-MM-YYYY
# df['PurchaseDate'] = df['PurchaseDate'].dt.strftime('%d-%m-%Y')
# df

In [ ]:
## Determine the refrence Date (e.g current date) ##

ref_date = data['PurchaseDate'].max()
ref_date

Timestamp('2023-06-10 00:00:00')

In [ ]:
# Calculate Recency
data['Recency'] = (ref_date - data['PurchaseDate']).dt.days
data

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,60
1,2188,2023-04-11,463.70,Product A,176819,London,60
2,4608,2023-04-11,80.28,Product A,340062,New York,60
3,2559,2023-04-11,221.29,Product A,239145,London,60
4,9482,2023-04-11,739.56,Product A,194545,Paris,60
...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,0
996,6669,2023-06-10,941.50,Product C,987025,New York,0
997,8836,2023-06-10,545.36,Product C,512842,London,0
998,1440,2023-06-10,729.94,Product B,559753,Paris,0


In [ ]:
# ### Calculate Frequency ##
# data['frequency'] = data.groupby('CustomerID')['OrderID'].transform('count')
# data

In [ ]:
## Calculate Frequency -- Using OrderID ##

frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)

data = data.merge(frequency_data, on = 'CustomerID', how ='left')

In [ ]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,60,1
1,2188,2023-04-11,463.70,Product A,176819,London,60,1
2,4608,2023-04-11,80.28,Product A,340062,New York,60,1
3,2559,2023-04-11,221.29,Product A,239145,London,60,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,60,1


In [ ]:
## Calculate Monetary -- Using Transaction Amount ##

monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'Monetary Value'}, inplace=True)

data = data.merge(monetary_data, on = 'CustomerID', how = 'left')

In [ ]:
data

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,Monetary Value
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,60,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,60,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,60,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,60,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,60,1,739.56
...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,0,1,759.62
996,6669,2023-06-10,941.50,Product C,987025,New York,0,1,941.50
997,8836,2023-06-10,545.36,Product C,512842,London,0,1,545.36
998,1440,2023-06-10,729.94,Product B,559753,Paris,0,1,729.94


**Calculating RFM Scores**

In [ ]:
# Define scoring criteria for each RFM value

recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

We assigned scores from 5 to 1 to calculate the recency score, where a higher score indicates a more recent purchase. It means that customers who have purchased more recently will receive higher recency scores.


We assigned scores from 1 to 5 to calculate the frequency score, where a higher score indicates a higher purchase frequency. Customers who made more frequent purchases will receive higher frequency scores.

To calculate the monetary score, we assigned scores from 1 to 5, where a higher score indicates a higher amount spent by the customer. **bold text**

In [ ]:
# Calculate RFM scores
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['Monetary Value'], bins=5, labels=monetary_scores)

**To calculate RFM scores, we used the pd.cut() function to divide recency, frequency, and monetary values into bins. We define 5 bins for each value and assign the corresponding scores to each bin.**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   Monetary Value      1000 non-null   float64       
 9   RecencyScore        1000 non-null   category      
 10  FrequencyScore      1000 non-null   category      
 11  MonetaryScore       1000 non-null   category      
dtypes: category(3), datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 74.0+ KB


**Once the scores are added to the data, you will notice that they are categorical variables. You can use the data.info() method to confirm this. So we need to convert their datatype into integers to use these scores further:**

In [ ]:
# Convert RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   Monetary Value      1000 non-null   float64       
 9   RecencyScore        1000 non-null   int64         
 10  FrequencyScore      1000 non-null   int64         
 11  MonetaryScore       1000 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(7), object(2)
memory usage: 93.9+ KB


**RFM Value Segmentatoin**

In [ ]:
# Calculate RFM score by combining the individual scores #
data['RFM_Scores'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']


In [ ]:
# Create RFM segments based on the RFM score #

segment_labels = ['Low-Value','Mid-Value','High-Value']
data['Value Segment']=pd.qcut(data['RFM_Scores'], q=3, labels = segment_labels)

**We divided RFM scores into three segments, namely “Low-Value”, “Mid-Value”, and “High-Value”. Segmentation is done using the pd.qcut() function, which evenly distributes scores between segments.**

In [ ]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,Monetary Value,RecencyScore,FrequencyScore,MonetaryScore,RFM_Scores,Value Segment
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,60,1,943.31,1,1,2,4,Low-Value
1,2188,2023-04-11,463.70,Product A,176819,London,60,1,463.70,1,1,1,3,Low-Value
2,4608,2023-04-11,80.28,Product A,340062,New York,60,1,80.28,1,1,1,3,Low-Value
3,2559,2023-04-11,221.29,Product A,239145,London,60,1,221.29,1,1,1,3,Low-Value
4,9482,2023-04-11,739.56,Product A,194545,Paris,60,1,739.56,1,1,2,4,Low-Value


**Now let’s have a look at the segment distribution:**

In [ ]:
# RFM Segment Distribution #

segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns= ['Value Segment','Count']
segment_counts

,Value Segment,Count
0,Low-Value,435
1,Mid-Value,386
2,High-Value,179


In [ ]:
## Creating Bar Chart ##

pastel_colors = px.colors.qualitative.Pastel

fig_segment_distrbn = px.bar(
    segment_counts, x='Value Segment', y='Count',
    color='Value Segment',color_discrete_sequence=pastel_colors,
    title='RFM Value Segment Distribution')

fig_segment_distrbn.update_layout(
    xaxis_title='RFM Value Segment',
    yaxis_title='Count',
    legend_title='Value Segment',
    width=1000,
    height=450)

fig_segment_distrbn.show()

*The RFM value segment represents the categorization of customers based on their RFM scores into groups such as “low value”, “medium value”, and “high value”. These segments are determined by dividing RFM scores into distinct ranges or groups, allowing for a more granular analysis of overall customer RFM characteristics.*

In [ ]:
# Now let’s create and analyze {[ RFM Customer Segments }] that are broader ##
# classifications based on the RFM scores.  ##

**RFM Customer Segments**

**These segments, such as “Champions”, “Potential Loyalists”, "At Risk Customers" , "Need Attention" and “Lost” provide a more strategic perspective on customer behaviour and characteristics in terms of recency, frequency, and monetary aspects.**

In [ ]:
# Create a new column for RFM Customer Segments
data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score

data.loc[data['RFM_Scores']>=10,'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Scores']>=7) & (data['RFM_Scores']<=9),'RFM Customer Segments'] = 'Potential Loyalist'
data.loc[(data['RFM_Scores']>=5) & (data['RFM_Scores']<7), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Scores']>=3) & (data['RFM_Scores']<5), 'RFM Customer Segments'] = 'Need Attention'
data.loc[data['RFM_Scores']<3, 'RFM Customer Segments'] = 'Lost'

data

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,Monetary Value,RecencyScore,FrequencyScore,MonetaryScore,RFM_Scores,Value Segment,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,60,1,943.31,1,1,2,4,Low-Value,Need Attention
1,2188,2023-04-11,463.70,Product A,176819,London,60,1,463.70,1,1,1,3,Low-Value,Need Attention
2,4608,2023-04-11,80.28,Product A,340062,New York,60,1,80.28,1,1,1,3,Low-Value,Need Attention
3,2559,2023-04-11,221.29,Product A,239145,London,60,1,221.29,1,1,1,3,Low-Value,Need Attention
4,9482,2023-04-11,739.56,Product A,194545,Paris,60,1,739.56,1,1,2,4,Low-Value,Need Attention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,0,1,759.62,5,1,2,8,High-Value,Potential Loyalist
996,6669,2023-06-10,941.50,Product C,987025,New York,0,1,941.50,5,1,2,8,High-Value,Potential Loyalist
997,8836,2023-06-10,545.36,Product C,512842,London,0,1,545.36,5,1,2,8,High-Value,Potential Loyalist
998,1440,2023-06-10,729.94,Product B,559753,Paris,0,1,729.94,5,1,2,8,High-Value,Potential Loyalist


###RFM Analysis ###

**Now let’s analyze the distribution of customers across different RFM customer segments within each value segment:**

In [ ]:
segment_product_counts = data.groupby(['Value Segment','RFM Customer Segments']).size().reset_index(name='Count')
segment_product_counts = segment_product_counts.sort_values('Count',ascending=False)
segment_product_counts

,Value Segment,RFM Customer Segments,Count
2,Low-Value,Need Attention,255
7,Mid-Value,Potential Loyalist,196
4,Mid-Value,At Risk Customers,190
0,Low-Value,At Risk Customers,180
11,High-Value,Potential Loyalist,145
9,High-Value,Champions,34
1,Low-Value,Champions,0
3,Low-Value,Potential Loyalist,0
5,Mid-Value,Champions,0
6,Mid-Value,Need Attention,0


In [ ]:
### Tree Map - RFM Customer Segment ###

fig_treemap = px.treemap(segment_product_counts,
                         path=['Value Segment','RFM Customer Segments'],
                         values='Count', color='Value Segment',
                         color_discrete_sequence = px.colors.qualitative.Pastel,
                         title='RFM Customer Segments by Value')

fig_treemap.update_layout(
    width=1000,
    height=650)

fig_treemap.show()

**Now let’s analyze the distribution of RFM values within the Champions segment:**

In [ ]:
# Filter the data to include only the customers in the Champions segment ##
champion_segment = data[data['RFM Customer Segments'] == 'Champions']
#champion_segment.value_counts('Value Segment')

In [ ]:
champion_segment

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,Monetary Value,RecencyScore,FrequencyScore,MonetaryScore,RFM_Scores,Value Segment,RFM Customer Segments
63,5557,2023-04-14,511.86,Product D,522866,London,57,3,2379.45,1,5,5,11,High-Value,Champions
180,5557,2023-04-22,996.98,Product D,765620,Tokyo,49,3,2379.45,1,5,5,11,High-Value,Champions
235,4804,2023-04-25,690.50,Product C,183336,Tokyo,46,3,2073.33,2,5,5,12,High-Value,Champions
249,7363,2023-04-26,262.45,Product C,778120,New York,45,3,1386.32,2,5,3,10,High-Value,Champions
291,7363,2023-04-28,188.02,Product C,726121,London,43,3,1386.32,2,5,3,10,High-Value,Champions
385,7363,2023-05-03,935.85,Product B,284654,London,38,3,1386.32,2,5,3,10,High-Value,Champions
446,9328,2023-05-06,788.22,Product A,268546,Tokyo,35,2,1710.17,3,3,4,10,High-Value,Champions
468,9871,2023-05-07,971.00,Product B,680493,New York,34,2,1820.53,3,3,4,10,High-Value,Champions
511,9514,2023-05-10,917.99,Product A,924758,Tokyo,31,2,1553.53,3,3,4,10,High-Value,Champions
525,7046,2023-05-10,950.75,Product C,710224,New York,31,2,1662.10,3,3,4,10,High-Value,Champions


In [ ]:
### Boxplots of RFM Values within Champions Segments ###

fig_boxplot = go.Figure()

fig_boxplot.add_trace(go.Box(y=champion_segment['RecencyScore'], name = "Recency"))
fig_boxplot.add_trace(go.Box(y=champion_segment['FrequencyScore'], name = "Frequency"))
fig_boxplot.add_trace(go.Box(y=champion_segment['MonetaryScore'], name = "Monetary"))

fig_boxplot.update_layout(
    title = 'Distribution of RFM Values within Champions Segment',
    yaxis_title = 'RFM Scores/Values',
    width = 1000,
    height = 450,
    showlegend = True
)

fig_boxplot.show()


In [ ]:
#  Now let’s analyze the correlation of the recency, frequency, and monetary
#  scores within the champions segment:

In [ ]:
correlation_matrix = champion_segment[['RecencyScore','FrequencyScore','MonetaryScore']].corr()
correlation_matrix

,RecencyScore,FrequencyScore,MonetaryScore
RecencyScore,1.000000,-0.708768,-0.514331
FrequencyScore,-0.708768,1.000000,0.510789
MonetaryScore,-0.514331,0.510789,1.000000


In [ ]:
# Visualize the correlation matrix using a heatmap ##

fig_heatmap = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    colorbar=dict(title='Correlation')
))

fig_heatmap.update_layout(
    title='Correlation Matrix of RFM Values within Champions Segment',
    width=1000,
    height=650
)

fig_heatmap.show()

**Now let’s have a look at the number of customers in all the segments:**

In [ ]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_customer_count = data['RFM Customer Segments'].value_counts()
segment_customer_count = segment_customer_count.reset_index()
segment_customer_count.columns = ['RFM Customer Segments','Count']
segment_customer_count

,RFM Customer Segments,Count
0,At Risk Customers,370
1,Potential Loyalist,341
2,Need Attention,255
3,Champions,34


In [ ]:
### Creating a Bar chart for the above RFM Customer Segments -- Counts ###

fig_bar = go.Figure(data=[go.Bar(
    x=segment_customer_count['RFM Customer Segments'],
    y=segment_customer_count['Count'],
    marker=dict(color=pastel_colors)
)])

fig_bar.update_layout(
    title='No.of Customers in Each RFM Segments',
    xaxis_title = 'RFM Customer Segments',
    yaxis_title = 'Count',
    width=1000,
    height=650
)

fig_bar.show()

In [ ]:
# Now let’s have a look at the recency, frequency,
# and monetary scores of all the segments:

In [ ]:
# Calculate the average Recency, Frequency, and Monetary scores for each segment #
average_segment_scores = data.groupby('RFM Customer Segments')[['RecencyScore','FrequencyScore','MonetaryScore']].mean().reset_index()
average_segment_scores


,RFM Customer Segments,RecencyScore,FrequencyScore,MonetaryScore
0,At Risk Customers,2.829730,1.064865,1.618919
1,Champions,3.852941,3.529412,3.617647
2,Need Attention,1.364706,1.000000,1.313725
3,Potential Loyalist,4.255132,1.346041,1.906158


In [ ]:
# Create a grouped bar chart to compare segment scores #

fig_group_bar = go.Figure()

# Add bars for Recency score #
fig_group_bar.add_trace(go.Bar(
    x=average_segment_scores['RFM Customer Segments'],
    y=average_segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color = 'rgb(158,202,225)'
))

# Add bars for Frequency score #
fig_group_bar.add_trace(go.Bar(
    x=average_segment_scores['RFM Customer Segments'],
    y=average_segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color = 'rgb(94,158,217)'
))

# Add bars for Monetary score #
fig_group_bar.add_traces(go.Bar(
    x=average_segment_scores['RFM Customer Segments'],
    y=average_segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color = 'rgb(44,102,148)'
))

fig_group_bar.update_layout(
    title = 'Comparison of RFM Segments based on RFM Scores',
    xaxis_title = 'RFM Customer Segments',
    yaxis_title = 'RFM Scores/Values',
    width = 1000,
    height = 450,
    barmode= 'group',
    showlegend = True
)

fig_group_bar.show()

**`**RFM Analysis is used to understand and segment customers based on their buying behaviour. **`**